In [1]:
import pandas# working with dataframes
import missingno #package to find and display na
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf#used for fitting models
import matplotlib.pyplot as plt
from plotnine import *
import stepwise as stepwise#make sure this file is located in the same folder as this script
from sklearn.metrics import roc_curve #AUC info
from sklearn.metrics import roc_auc_score# AUC score
from sklearn.metrics import accuracy_score #accuracy performance measure
from sklearn.metrics import recall_score# sensitivity performance measure
from sklearn.metrics import precision_score# precision performance measure
from sklearn.metrics import confusion_matrix# confusion matrix
from mizani.formatters import percent_format #to have % on axis in plotnine
import sys #maxsize
import seaborn as sns# heatmap
import datetime# working with dates
from sklearn.model_selection import train_test_split

In [2]:
#Read in dataframes
summerfintrx = pandas.read_csv('summerfintrx.csv',parse_dates=['Date'])
summersesstrx = pandas.read_csv('summersesstrx.csv',parse_dates=['Date'])
fallfintrx = pandas.read_csv('fallfintrx.csv',parse_dates=['Date'])
fallsesstrx = pandas.read_csv('fallsesstrx.csv',parse_dates=['Date'])
customerdata = pandas.read_csv('customerdata.csv',parse_dates=['Registrationdate'])

#convert datetime to date (we don't need the hour)
summerfintrx['Date'] = pandas.to_datetime(summerfintrx['Date']).dt.date
summersesstrx['Date'] = pandas.to_datetime(summersesstrx['Date']).dt.date
fallfintrx['Date'] = pandas.to_datetime(fallfintrx['Date']).dt.date
fallsesstrx['Date'] = pandas.to_datetime(fallsesstrx['Date']).dt.date
customerdata['Registrationdate'] = pandas.to_datetime(customerdata['Registrationdate']).dt.date

In [3]:
summersesstrx["Date"].max()

datetime.date(2022, 8, 30)

In [4]:
fallsesstrx["Date"].max()

datetime.date(2022, 12, 30)

In [5]:
summersesstrx.isnull().sum()

SessionID     0
Date          0
CustomerID    0
Pokestops     0
Gyms          0
Raids         0
Social        0
Pokemons      0
Distance      0
Duration      0
Experience    0
dtype: int64

In [6]:
fallsesstrx.isnull().sum()

SessionID     0
Date          0
CustomerID    0
Pokestops     0
Gyms          0
Raids         0
Social        0
Pokemons      0
Distance      0
Duration      0
Experience    0
dtype: int64

In [7]:
customerdata

,CustomerID,CustomerType,Registrationdate,Gender,Age,fallbonus,Income
0,1,2,2021-01-01,0,8,0,0
1,2,0,2022-04-24,0,29,0,2
2,3,1,2022-04-18,0,21,0,0
3,4,0,2019-11-17,0,16,0,1
4,5,0,2022-01-12,0,29,1,0
...,...,...,...,...,...,...,...
49995,49996,2,2018-12-15,0,56,1,0
49996,49997,1,2021-09-17,0,52,0,1
49997,49998,3,2019-10-26,0,37,0,2
49998,49999,0,2022-02-27,0,13,1,0


In [8]:
summersesstrx

,SessionID,Date,CustomerID,Pokestops,Gyms,Raids,Social,Pokemons,Distance,Duration,Experience
0,1,2022-06-21,1,8,10,1,1,10,1.342066,69.426156,4234
1,2,2022-08-27,1,13,5,0,0,10,0.897713,113.545351,2239
2,3,2022-07-23,1,12,13,0,5,9,1.046671,69.182332,4754
3,4,2022-07-22,1,10,6,1,0,19,2.193635,109.283062,3869
4,5,2022-05-07,1,10,9,1,5,21,1.840136,70.036803,5534
...,...,...,...,...,...,...,...,...,...,...,...
379557,379558,2022-06-02,50000,20,7,0,0,18,6.465864,22.777787,3946
379558,379559,2022-07-26,50000,22,10,0,0,13,6.257543,68.159570,4375
379559,379560,2022-06-14,50000,19,9,0,0,15,11.587589,70.789062,4658
379560,379561,2022-05-04,50000,14,7,0,0,12,13.403382,57.314424,4040


In [9]:
fallsesstrx

,SessionID,Date,CustomerID,Pokestops,Gyms,Raids,Social,Pokemons,Distance,Duration,Experience
0,1,2022-11-19,1,8,2,0,0,17,1.425847,76.208651,1792
1,2,2022-10-08,1,12,12,0,1,8,1.237647,105.494152,3723
2,3,2022-09-03,1,11,12,0,0,8,1.099179,65.876782,3459
3,4,2022-09-28,1,12,11,0,0,8,1.645335,93.312920,3364
4,5,2022-10-16,1,10,7,0,0,19,1.308033,51.978233,2980
...,...,...,...,...,...,...,...,...,...,...,...
297223,297224,2022-09-26,49999,19,2,0,0,28,10.110494,67.213172,3761
297224,297225,2022-10-16,49999,22,3,0,0,13,7.396345,98.167656,3089
297225,297226,2022-11-23,49999,14,12,0,0,11,15.965344,63.701975,5246
297226,297227,2022-10-28,50000,16,5,0,0,12,7.318495,60.284210,3131


In [10]:
summerfintrx

,TransactionID,Date,CustomerID,ProductID,SessionID,Value
0,1,2022-05-21,1,2,6,9.99
1,2,2022-07-01,6,0,54,2.99
2,3,2022-05-10,10,3,90,25.00
3,4,2022-06-22,14,3,114,25.00
4,5,2022-08-24,14,0,117,2.99
...,...,...,...,...,...,...
21859,21860,2022-07-27,49989,1,379478,4.99
21860,21861,2022-06-24,49989,1,379482,4.99
21861,21862,2022-07-08,49992,1,379498,4.99
21862,21863,2022-07-26,49995,1,379525,4.99


In [11]:
summersesstrx["Date"].min()

datetime.date(2022, 5, 1)

In [12]:
summersesstrx["Date"].max()

datetime.date(2022, 8, 30)